In [1]:
# import some useful libraries
import glob, nltk, joblib
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras import metrics

from PIL import Image
from tqdm.notebook import tqdm

from sklearn.feature_extraction.text import CountVectorizer

from music21 import midi
from plugins.midi2img import midi2img
from plugins.img2midi import img2midi
from IPython.display import clear_output

In [2]:
DATA_PATH = '../data/dataset'

In [3]:
# load in the metadata
# create a list of captions that concatenate the piece description and arousal
# lower case te caption list
midi_meta = pd.read_csv(f'{DATA_PATH}/commu_meta_caption.csv')
# midi_meta['caption_list'] = midi_meta['piece_description'].str.lower()+ ". " + midi_meta['piece_arousal'].str.lower()
midi_meta

,Unnamed: 0,audio_key,chord_progressions,pitch_range,num_measures,bpm,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id,caption_list
0,0,"['a', 'minor']","[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",['mid'],8,120,cinematic,"['main', 'melody']","['string', 'ensemble']",standard,4-Apr,101,102,train,commu00001,"a, minor, mid, main, melody, cinematic, string..."
1,1,"['c', 'major']","[['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Dm'...","['mid', 'low']",8,80,newage,['accompaniment'],"['acoustic', 'piano']",standard,4-Apr,23,30,train,commu00002,"c, major, mid, low, accompaniment, newage, aco..."
2,2,"['a', 'minor']","[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...","['mid', 'high']",8,150,cinematic,['riff'],"['string', 'violin']",standard,4-Apr,123,127,train,commu00003,"a, minor, mid, high, riff, cinematic, string, ..."
3,3,"['c', 'major']","[['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Gm'...",['mid'],8,110,cinematic,['pad'],['choir'],standard,4-Apr,45,46,train,commu00004,"c, major, mid, pad, cinematic, choir"
4,4,"['a', 'minor']","[['Am', 'Am', 'Am', 'Am', 'Em', 'Em', 'Em', 'E...","['mid', 'low']",4,60,cinematic,['pad'],"['acoustic', 'piano']",standard,4-Apr,21,22,train,commu00005,"a, minor, mid, low, pad, cinematic, acoustic, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11139,11139,"['c', 'major']","[['Fmaj7', 'Fmaj7', 'Fmaj7', 'Fmaj7', 'Fmaj7',...","['mid', 'high']",8,110,cinematic,['pad'],"['synth', 'pad']",standard,4-Apr,83,84,val,commu11140,"c, major, mid, high, pad, cinematic, synth, pad"
11140,11140,"['c', 'major']","[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...","['mid', 'low']",8,110,cinematic,['riff'],"['acoustic', 'guitar']",standard,4-Apr,51,55,val,commu11141,"c, major, mid, low, riff, cinematic, acoustic,..."
11141,11141,"['a', 'minor']","[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...","['very', 'low']",8,145,cinematic,['bass'],"['string', 'ensemble']",standard,4-Apr,109,112,val,commu11142,"a, minor, very, low, bass, cinematic, string, ..."
11142,11142,"['a', 'minor']","[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",['low'],8,140,cinematic,['accompaniment'],"['string', 'cello']",standard,4-Apr,55,115,val,commu11143,"a, minor, low, accompaniment, cinematic, strin..."


In [4]:
# read in music and prepare it in a list of notes
midis = []
for file in tqdm(glob.glob(f"{DATA_PATH}/commu_midi/train/raw/*.mid")):
    midis.append(file)

for file in tqdm(glob.glob(f"{DATA_PATH}/commu_midi/val/raw/*.mid")):
    midis.append(file)

  0%|          | 0/10381 [00:00<?, ?it/s]

  0%|          | 0/763 [00:00<?, ?it/s]

In [5]:
# convert MIDI to image
# declare output path
output_path = f"{DATA_PATH}/commu_img/train"

# instantiate a midi2img converter object
midi2img_obj = midi2img(output_path)

# for each midi file, convert it to image
for midi in tqdm(midis):
    data, metadata = midi2img_obj.convert_to_image(midi)
    for img_key in data.keys():
        try:
            # if img_key[-1:] != "0": continue
            img_path = output_path+"/"+midi.split("/")[-1].replace(".mid",f"_{img_key}.png")
            img = Image.open(img_path)
            basewidth = 106
            hsize = 106
            img = img.resize((basewidth,hsize), Image.ANTIALIAS)
            img.save(img_path)
        except Exception as e:
            print("Error in converting: ", e)
            pass

  0%|          | 0/11144 [00:00<?, ?it/s]

C:\Users\ktrin\AppData\Local\Temp\ipykernel_4048\2302571671.py:18: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((basewidth,hsize), Image.ANTIALIAS)


Error in converting:  [Errno 2] No such file or directory: '../data/dataset/commu_img/train/raw\\commu00001_Piano_2.png'
Error in converting:  [Errno 2] No such file or directory: '../data/dataset/commu_img/train/raw\\commu00002_Piano_2.png'
Error in converting:  [Errno 2] No such file or directory: '../data/dataset/commu_img/train/raw\\commu00003_Piano_2.png'
Error in converting:  [Errno 2] No such file or directory: '../data/dataset/commu_img/train/raw\\commu00004_Piano_2.png'
Error in converting:  [Errno 2] No such file or directory: '../data/dataset/commu_img/train/raw\\commu00005_Piano_1.png'
Error in converting:  [Errno 2] No such file or directory: '../data/dataset/commu_img/train/raw\\commu00006_Piano_2.png'
Error in converting:  [Errno 2] No such file or directory: '../data/dataset/commu_img/train/raw\\commu00007_Piano_2.png'
Error in converting:  [Errno 2] No such file or directory: '../data/dataset/commu_img/train/raw\\commu00008_Piano_1.png'
Error in converting:  [Errno 2] 

In [6]:
# Dataset Construction
def access_images(path, metadata, piece_name='piece_name', num_images=float("inf")):
    """
        Fetch a defined number of images from a directory
        and return their normalized pixels arrays
        Inputs:
            path: path to the directory containing the images
            img_list: list of image names
            num_images: number of images to fetch
        Output:
            image_arr: pixel array of images
            img: the original image
    """
    pixels = {}

    # shuffle the metadata to get a random sample
    if num_images != float("inf"):
        metadata = metadata.sample(n=num_images).reset_index(drop=True)
    else:
        metadata = metadata.sample(frac=1).reset_index(drop=True)

    i = 0
    # construct a dictionary of empty lists with the song name
    for piece in tqdm(metadata[piece_name].unique()):
        if i >= num_images:
            break
        
        # construct the file path for the image
        file = f"{path}/{piece}_Piano_0.png"
        try:
            # Normalize the image
            img = Image.open(file, 'r')
            img = img.convert('1')
            pix = np.array(img.getdata())
            pix = pix.astype('float32')
            pix /= 255.0

            # pixels.append(pix.reshape(106,106,1))
            # once a valid image is found, break the loop
            if piece not in pixels.keys():

                pixels[piece] = [pix.reshape(106,106,1)]
            else:
                pixels[piece].append(pix.reshape(106,106,1))

        except Exception as e:
            # print("Error in fetching: ", e)
            pass
        i+=1
    return pixels

img_dir = f"{DATA_PATH}/commu_img/train/raw"
# get train data only
midi_meta = midi_meta[midi_meta['split_data'] == 'train']
pixels_arr = access_images(img_dir, midi_meta, piece_name='id')
len(pixels_arr.keys())

  0%|          | 0/10381 [00:00<?, ?it/s]

10246

In [7]:
# # Dataset Construction
# def access_images(path, metadata, piece_name='piece_name', num_images=float("inf")):
#     """
#         Fetch a defined number of images from a directory
#         and return their normalized pixels arrays
#         Inputs:
#             path: path to the directory containing the images
#             img_list: list of image names
#             num_images: number of images to fetch
#         Output:
#             image_arr: pixel array of images
#             img: the original image
#     """
#     pixels = {}

#     # shuffle the metadata to get a random sample
#     metadata = metadata.sample(n=num_images).reset_index(drop=True)

#     # fetch all images from the directory
#     i = 0
#     # construct a dictionary of empty lists with the song name
#     for piece in tqdm(metadata[piece_name].unique()):
#         # if row['piece_name'] not in pixels.keys():
#         #     pixels[row['piece_name']] = []
#         if i >= num_images:
#             break

#         for file in (glob.glob(f"{path}/*.png")):
#             try:
#                 img_path = file.split("/")[-1]
#                 # ignore the images that are not the first image
#                 if img_path[-5:-4] != '0':
#                     continue

#                 # Normalize the image
#                 img = Image.open(file, 'r')
#                 img = img.convert('1')
#                 pix = np.array(img.getdata())
#                 pix = pix.astype('float32')
#                 pix /= 255.0

#                 # pixels.append(pix.reshape(106,106,1))
#                 # once a valid image is found, break the loop
#                 if img_path.find(piece) != -1:
#                     if piece not in pixels.keys():
#                         pixels[piece] = [pix.reshape(106,106,1)]
#                     else:
#                         pixels[piece].append(pix.reshape(106,106,1))
#                     break

#             except Exception as e:
#                 print("Error in fetching: ", e)
#                 pass

#         i += 1
#     return pixels


# img_dir = f"{DATA_PATH}/commu_img/train/raw"
# # get train data only
# midi_meta = midi_meta[midi_meta['split_data'] == 'train']
# pixels_arr = access_images(img_dir, midi_meta, piece_name='id', num_images=2048)
# pixels_arr.keys()
# # np.unique(pixels_arr['commu00001'])

In [8]:
# create a dataframe of image arrays and piece names
join_col = 'id'
sound_image = pd.DataFrame(columns=[join_col, 'image_array'])#from_dict(pixels_arr, orient='index').reset_index()
sound_image[join_col] = pixels_arr.keys()
sound_image['image_array'] = pixels_arr.values()
sound_image

# merge with metadata
midi_meta = midi_meta.merge(sound_image, on=join_col, how='left').dropna(subset=[join_col])


# create image-caption pairs
training_set = []
for i, row in midi_meta.iterrows():
    caption = row['caption_list']
    images = row['image_array']
    id = row['id']
    try:
        for image in images:
            training_set.append((id, image, caption))
    except:
        pass



In [9]:
training_set[0][1].shape,training_set


((106, 106, 1),
 [('commu00001', array([[[0.],
           [0.],
           [0.],
           ...,
           [0.],
           [0.],
           [0.]],
   
          [[0.],
           [0.],
           [0.],
           ...,
           [0.],
           [0.],
           [0.]],
   
          [[0.],
           [0.],
           [0.],
           ...,
           [0.],
           [0.],
           [0.]],
   
          ...,
   
          [[0.],
           [0.],
           [0.],
           ...,
           [0.],
           [0.],
           [0.]],
   
          [[0.],
           [0.],
           [0.],
           ...,
           [0.],
           [0.],
           [0.]],
   
          [[0.],
           [0.],
           [0.],
           ...,
           [0.],
           [0.],
           [0.]]], dtype=float32), 'a, minor, mid, main, melody, cinematic, string, ensemble'),
  ('commu00002',
   array([[[0.],
           [0.],
           [0.],
           ...,
           [0.],
           [0.],
           [0.]],
   

In [10]:
# save data as joblib
joblib.dump(training_set, f'{DATA_PATH}/training_set.joblib')

['../data/dataset/training_set.joblib']